In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
import pandas as pd
file_path = '/content/drive/My Drive/complaints.csv'
df =pd.read_csv(file_path)

<ipython-input-2-de73d907ca24>:3: DtypeWarning: Columns (16) have mixed types. Specify dtype option on import or set low_memory=False.
  df =pd.read_csv(file_path)


In [3]:
df.head(2)

,Date received,Product,Sub-product,Issue,Sub-issue,Consumer complaint narrative,Company public response,Company,State,ZIP code,Tags,Consumer consent provided?,Submitted via,Date sent to company,Company response to consumer,Timely response?,Consumer disputed?,Complaint ID
0,2021-02-15,"Money transfer, virtual currency, or money ser...",Domestic (US) money transfer,Other transaction problem,NaN,"ON XX/XX/2021, THERE WERE MULTIPLE FRAUDULENT ...",NaN,"Block, Inc.",TX,75035,NaN,Consent provided,Web,2021-02-15,Closed with explanation,Yes,NaN,4140721
1,2021-08-27,"Money transfer, virtual currency, or money ser...",Domestic (US) money transfer,Other transaction problem,NaN,On Sunday XX/XX/2021 on or about XXXX XXXX I ...,NaN,"Block, Inc.",CA,94565,Servicemember,Consent provided,Web,2021-08-27,Closed with explanation,Yes,NaN,4668568


In [4]:
df['Product'].unique()

array(['Money transfer, virtual currency, or money service',
       'Credit reporting or other personal consumer reports',
       'Credit card or prepaid card', 'Debt collection', 'Credit card',
       'Checking or savings account', 'Mortgage', 'Vehicle loan or lease',
       'Prepaid card', 'Student loan',
       'Payday loan, title loan, personal loan, or advance loan',
       'Debt or credit management',
       'Credit reporting, credit repair services, or other personal consumer reports',
       'Bank account or service', 'Credit reporting', 'Consumer Loan',
       'Payday loan, title loan, or personal loan', 'Money transfers',
       'Payday loan', 'Other financial service', 'Virtual currency'],
      dtype=object)

In [5]:
classes = ['Credit reporting, credit repair services, or other personal consumer report',
           'Debt collection',
           'Consumer Loan',
           'Mortgage']

In [6]:
sub_df = df[df['Product'].isin(classes)]

In [7]:
sub_df

,Date received,Product,Sub-product,Issue,Sub-issue,Consumer complaint narrative,Company public response,Company,State,ZIP code,Tags,Consumer consent provided?,Submitted via,Date sent to company,Company response to consumer,Timely response?,Consumer disputed?,Complaint ID
21,2024-05-15,Debt collection,Credit card debt,Attempts to collect debt not owed,Debt is not yours,NaN,NaN,PORTFOLIO RECOVERY ASSOCIATES INC,CO,80634,NaN,NaN,Web,2024-05-15,Closed with explanation,Yes,NaN,9004583
45,2024-05-29,Debt collection,Telecommunications debt,Written notification about debt,Didn't receive enough information to verify debt,NaN,NaN,"Credence Resource Management, LLC",TX,774XX,NaN,NaN,Web,2024-05-29,In progress,Yes,NaN,9121091
47,2024-05-29,Debt collection,Telecommunications debt,Written notification about debt,Didn't receive notice of right to dispute,NaN,Company believes it acted appropriately as aut...,"Diversified Adjustment Service, Inc.",FL,34997,NaN,Consent not provided,Web,2024-05-29,Closed with explanation,Yes,NaN,9121097
52,2024-05-29,Debt collection,Other debt,Written notification about debt,Notification didn't disclose it was an attempt...,NaN,NaN,"EQUIFAX, INC.",TX,79765,NaN,NaN,Web,2024-05-29,In progress,Yes,NaN,9121105
72,2024-05-29,Debt collection,Other debt,Written notification about debt,Notification didn't disclose it was an attempt...,NaN,NaN,"EQUIFAX, INC.",IN,46320,NaN,NaN,Web,2024-05-29,In progress,Yes,NaN,9121266
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5336447,2022-03-01,Debt collection,Other debt,Written notification about debt,Didn't receive enough information to verify debt,NaN,NaN,CONTRACT CALLERS INC,MD,20877,NaN,Consent not provided,Web,2022-03-01,Closed with explanation,Yes,NaN,5276304
5336448,2022-03-11,Debt collection,I do not know,Attempts to collect debt not owed,Debt is not yours,NaN,NaN,Resurgent Capital Services L.P.,LA,70075,NaN,NaN,Phone,2022-03-11,Closed with explanation,Yes,NaN,5316882
5336450,2022-03-11,Debt collection,Other debt,Attempts to collect debt not owed,Debt was result of identity theft,NaN,NaN,"Security Credit Services, LLC",TX,77090,NaN,Consent not provided,Web,2022-03-11,Closed with explanation,Yes,NaN,5309156
5336461,2023-08-08,Debt collection,Credit card debt,Written notification about debt,Didn't receive enough information to verify debt,NaN,NaN,Resurgent Capital Services L.P.,MN,55408,NaN,Consent not provided,Web,2023-08-08,Closed with explanation,Yes,NaN,7371480


In [8]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import numpy as np
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split


In [9]:
label_encoder = LabelEncoder()
labels = label_encoder.fit_transform(sub_df['Product'])

# NOTE I AM ONLY TAKING THE COLUMN 'ISSUE' AS FEATURE FOR NOW

sub_df['Issue'] = sub_df['Issue'].astype(str)#map(lambda x: str(x))
tokenized_comments = [comment.split() for comment in sub_df['Issue']]

<ipython-input-9-00bb76042b1b>:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  sub_df['Issue'] = sub_df['Issue'].astype(str)#map(lambda x: str(x))


In [10]:
vocab = {word for comment in tokenized_comments for word in comment}
word_to_idx = {word: idx for idx, word in enumerate(vocab, 1)}
word_to_idx['<PAD>'] = 0

max_len = max(len(comment) for comment in tokenized_comments)
sequences = [[word_to_idx[word] for word in comment] for comment in tokenized_comments]
padded_sequences = [seq + [0] * (max_len - len(seq)) for seq in sequences]
X_train, X_test, y_train, y_test = train_test_split(padded_sequences, labels, test_size=0.2, random_state=42)

In [11]:
class TextDataset(Dataset):
    def __init__(self, sequences, labels):
        self.sequences = sequences
        self.labels = labels

    def __len__(self):
        return len(self.sequences)

    def __getitem__(self, idx):
        return torch.tensor(self.sequences[idx], dtype=torch.long), torch.tensor(self.labels[idx], dtype=torch.long)

train_dataset = TextDataset(X_train, y_train)
test_dataset = TextDataset(X_test, y_test)

train_loader = DataLoader(train_dataset, batch_size=2, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=2, shuffle=False)

In [12]:
class RNNModel(nn.Module):
    def __init__(self, vocab_size, embed_size, hidden_size, output_size):
        super(RNNModel, self).__init__()
        self.embedding = nn.Embedding(vocab_size, embed_size, padding_idx=0)
        self.rnn = nn.RNN(embed_size, hidden_size, batch_first=True)
        self.fc = nn.Linear(hidden_size, output_size)

    def forward(self, x):
        x = self.embedding(x)
        out, _ = self.rnn(x)
        out = out[:, -1, :]  # Take the output from the last time step
        out = self.fc(out)
        return out

vocab_size = len(word_to_idx)
embed_size = 8
hidden_size = 16
output_size = len(label_encoder.classes_)

model = RNNModel(vocab_size, embed_size, hidden_size, output_size)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)


In [13]:
from tqdm import tqdm

In [14]:
num_epochs = 10

for epoch in range(num_epochs):
    model.train()
    for sequences, labels in tqdm(train_loader):
        outputs = model(sequences)
        loss = criterion(outputs, labels)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

    print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {loss.item():.4f}')


100%|██████████| 403103/403103 [17:41<00:00, 379.63it/s]


Epoch [1/10], Loss: 0.0000


100%|██████████| 403103/403103 [17:55<00:00, 374.73it/s]


Epoch [2/10], Loss: 0.0000


100%|██████████| 403103/403103 [17:56<00:00, 374.58it/s]


...

Epoch [8/10], Loss: 0.0000


100%|██████████| 403103/403103 [17:57<00:00, 374.12it/s]


Epoch [9/10], Loss: 0.0000


100%|██████████| 403103/403103 [17:58<00:00, 373.60it/s]

Epoch [10/10], Loss: 0.0000


In [15]:
model.eval()
with torch.no_grad():
    correct = 0
    total = 0
    for sequences, labels in test_loader:
        outputs = model(sequences)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

    print(f'Accuracy: {100 * correct / total:.2f}%')


Accuracy: 100.00%
